# Dust-line Mass calibration

# Formulae

The surface density estimate after Powell et al. 2017 but accounting properly for pre-factors is

$$\Sigma_\mathsf{g} = \frac{\gamma}{2} \frac{t_\mathsf{disk}\, v_0 \, \rho_\mathsf{s} \, \lambda}{r}$$

Original Powell et al. 2019

$$\Sigma_\mathsf{g} = \frac{2.5}{2 \pi} \frac{t_\mathsf{disk}\, v_0 \, \rho_\mathsf{s} \, \lambda}{r}$$

Result: Powell pre-factor is $\sim 0.4$, our pre-factor is $\sim 1.4$, so we are about 3.5 times more massive. This does agree roughly with how much we over-predict the mass. Looks like they dropped about the right amount of pre factors to get the right result.

Our logit part reads

$$
y = 1 - \frac{A}{1 + \exp\left(-\frac{x - x_0}{\Delta x}\right)}
$$

which goes from 1 for small $x$ to $1-A$ for large $x$. 

If we want to be $N=90\%$ towards the right limit, we need to go to

$$x_r = x_0 + \Delta x  \log \left(\frac{N}{1 - N}\right)$$

We picked $\Delta x$ to be $0.1 \, x_0$, so the dust line would be at

$$x = x_0 (1 + 0.1 \ln(9)) \simeq 1.22 x_0$$

If we want to be at $N=90\%$ in log space, we need to go to

$$
x_r = x_0 + \Delta x \ln\left(\frac{A}{(1 - A)^N + A - 1} - 1\right)
$$

**Dust and gas surface densities from simulation and Powell method**

\begin{align}
v_0 &= \frac{c_s^2}{2  V_k}\\
\Sigma_d &= \frac{\lambda}{2 f} \frac{v_0}{V_k} \, \rho_s \, \gamma\\
&= \frac{a}{10 cm} \left(\frac{\frac{h}{r}}{0.1}\right)^2
\end{align}

# Todo:

- try at different times
- randomize times
- fix missing low(?) alpha analysis.
- try to get a reasonable expression/fit of the error curve for proper calibration.
- try to find out why some simulations are off by *a lot*
- try the *original Powell* method: fitting just truncated power-laws and then fitting a self-similar profile.
- find out why the mass is still well reproduced even if the surface densities are off by *a lot*

# Setup

In [ ]:
from pathlib import Path

import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import corner

import dipsy
import estimator
import analysis_mass

import dipsy.data_frame_functions as ddff

M_sun = dipsy.cgs_constants.M_sun
year = dipsy.cgs_constants.year
au = dipsy.cgs_constants.au
year = dipsy.cgs_constants.year
lams = analysis_mass.lams

Define constants

In [ ]:
flux_fraction = 0.68

In [ ]:
fname = 'dustlines.hdf5'

In [ ]:
t_disk = 3e6 * year

# Analyze Grid

## Loading file and parameters

In [ ]:
df = pd.DataFrame(dipsy.utils.read_from_hdf5('dustlines_mass_3.0e+06yr_f3.hdf5'))

we add a column `error` to select based on its value

In [ ]:
df['error'] = df.M_est/df.M_gas

Set parameters

In [ ]:
param_names = ['v_frag', 'alpha', 'Mdisk', 'r_c', 'M_star']

# We also define the "nice names" of each parameter for the labels

param_label = {
    'alpha': r'$\alpha$',
    'v_frag': r'$v_\mathsf{frag}$',
    'Mdisk': r'$M_\mathsf{disk}$',
    'M_star': r'$M_\star$',
    'r_c': r'$r_\mathsf{c}$',
    'M_gas': '$M_\mathrm{gas}$ [$M_\star$]',
    'M_est': r'$M_\mathrm{gas,estim.}$'
}

param_values = ddff.get_param_values(df, param_names)

In [ ]:
print('we have the following parameters:')
for key, value in param_values.items():
    print((key + f'({len(value)}):').ljust(15), end='')
    print(', '.join([f'{v:.2g}' for v in value]))

## Heatmaps

Plotting the relation between estimate and true gas mass.

In [ ]:
for k in ['M_est', 'M_lbp_med']:

    f, ax = plt.subplots(figsize=(5,5))
    ax.set_aspect('equal')
    ax.set_xlim(1e-4, .3)
    ax.set_ylim(1e-4, 1e1)

    ax.hexbin(df['M_gas'] / M_sun, df[k]/ M_sun, gridsize=(50,50),
              xscale='log', yscale='log', vmax=20,
              rasterized=False, cmap='cividis',
              linewidths=0.01, edgecolor='face')

    ax.plot([1e-4, 1e1], [1e-4, 1e1], 'w--')

    _x = np.array([1e-4, 1e1])
    _y = 0.15 * (_x/0.1)**0.8

    ax.loglog(_x, _y, 'r-')
    ax.set_xlabel(param_label['M_gas'])
    ax.set_ylabel(param_label['M_est']);

    f.savefig(f'heat_value_{k}.pdf', dpi=300, transparent=True, bbox_inches='tight')

Plotting the same but in terms of error = deviation from true solution.

In [ ]:
x = df['M_gas']
y = df['M_est']/df['M_gas']

f, ax = plt.subplots(figsize=(5,5))
ax.set_xlim(1e-4, .3)
ax.set_ylim(df.error.min(), 1e4)

ax.hexbin(x / M_sun, y, gridsize=50,
          xscale='log', yscale='log', vmax=20,
          rasterized=False, cmap='cividis',
          linewidths=0.2, edgecolor='face')
ax.axhline(1.0, c='w', ls='--')

ax.set_xlabel(param_label['M_gas'])
ax.set_ylabel(r'$M_\mathrm{gas,estim.} / M_\mathrm{gas}$');

f.savefig('heat_error.pdf', dpi=300, transparent=True, bbox_inches='tight')

In [ ]:
e_min = df.error.min()
e_max = df.error.max()
M_min = df.M_gas.min()
M_max = df.M_gas.max()
fact = np.log(e_max/e_min) / np.log(M_max/M_min)
n_bin = 40

param_values2 = {
    'error': np.logspace(-1, 4, int((n_bin+1) * fact)),
    'M_gas': np.logspace(-4, np.log10(.3), (n_bin+1)) * M_sun   
}

param_interfaces2 = ddff.make_interfaces(param_values2)

In [ ]:
counts, x_c, y_c = np.histogram2d(
    df.M_gas,
    df.error,
    bins=[
        param_interfaces2['M_gas'],
        param_interfaces2['error'],
    ])

In [ ]:
f, ax = plt.subplots()
ax.loglog(param_values2['M_gas'] / M_sun,
          np.convolve(
              (param_values2['error'] * counts).mean(1),
              np.ones(5)/5, mode='same')
         )
ax.loglog(param_values2['M_gas'] / M_sun,
          #np.convolve(
              np.median((param_values2['error'] * counts), 1),
          #    np.ones(5)/5, mode='same')
         )
ax.set_ylim(bottom=2e-2)

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
ax.set_aspect('equal')
ax.pcolormesh(x_c/M_sun, y_c, counts.T, vmax=counts.max() * 0.5)
ax.axhline(1, c='w', ls='--')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(param_label['M_gas'])
ax.set_ylabel('error');

## Histograms

apply the filter function

These two plots that follow show the fraction of good estimates per bin. If the value is 100% in a bin, then all those simulations were well estimated. For testing this, one can set
```
df['error'] = 1
```
before the filter function is called (i.e. before `res` is calculated).

In [ ]:
f = lambda row: ddff.filter_function(row, error=[0.3, 3])
res = df[df.apply(f, axis=1)].copy()
print(f'found {len(res)} matching simulations ({len(res) / len(df):.1%})')

In [ ]:
f = ddff.histogram_corner(res, param_values, param_label=param_label,
                          percent='per_bin', n_total=len(df), vmax=100)
f.savefig('histogram_corner.pdf', dpi=300, transparent=True, bbox_inches='tight')

Let's check the parameters of the simulations that have large errors.

In [ ]:
f = lambda row: ddff.filter_function(row, error=[3, np.inf])
high_error = df[df.apply(f, axis=1)].copy()
print(f'found {len(res)} matching simulations ({len(high_error) / len(df):.1%})')

f = ddff.histogram_corner(high_error, param_values, param_label=param_label,
                          percent='per_bin', n_total=len(df), vmax=100)
f.savefig('histogram_corner.pdf', dpi=300, transparent=True, bbox_inches='tight')

Now filter one of the simulations which tend to have the highest errors

In [ ]:
f = lambda row: ddff.filter_function(row, error=[30, np.inf], alpha=1e-2, v_frag=100., M_star=param_values['M_star'][0])
test = df[df.apply(f, axis=1)].copy()
print(f'found {len(test)} matching simulations ({len(test) / len(df):.1%})')
test_idx = np.random.choice(test.index)
key = f'{test_idx:07d}'

<hr>
<h1 style="color:#D30">Testing Area</h1>

## Load a simulation

Get number of simulations

In [ ]:
with h5py.File(fname, 'r') as fh:
    n_sim = len(fh)

**Pick a random model**

find one with $ \alpha = 10^{-3}$

In [ ]:
# to select a random simulation out of as subset of all simulations
# sims = pd.DataFrame(dipsy.utils.read_from_hdf5(fname))
# key = f'{np.random.choice(np.random.choice(sims[sims.alpha==3e-3].index)):07d}'

In [ ]:
# random key
key = f'{int(n_sim * np.random.rand()):07d}'

# previous issues:
# key = '0006370'
# key = '0006780' # disk depleted
# key = '0000011' # disk depleted
# key = '0001184'
print(key)

**load the simulation**

In [ ]:
# actual simulation
sim = dipsy.utils.read_from_hdf5(fname, key)
it = sim['time'].searchsorted(t_disk)

## Fitting for dust lines

In [ ]:
settings = {
    'time': t_disk,    
    'q' : 3.5,
    'flux_fraction': 0.68,
    'fname_in' : fname,
    'fname_out' : 'test.hdf5',
    'opac' : dipsy.Opacity(input='ricci_compact.npz'),
    'fct_nr' : 3,
}

In [ ]:
from importlib import reload
import estimator
reload(estimator)
import analysis_mass
reload(analysis_mass)

In [ ]:
res = analysis_mass.parallel_analyze(key, settings=settings, debug=True, progress=True, n_burnin=400, n_steps=1500)

**Process the results**

In [ ]:
a = lams / (2 * np.pi)

p_gas = [res[k] for k in ['N_g', 'rc_g', 'p_g']]
p_dust = [res[k] for k in ['N_d', 'rc_d', 'p_d']]

masses_g = res['masses_g']
masses_d = res['masses_d']
sampler_g = res['sampler_g']
sampler_d = res['sampler_d']

y_dust = res['sig_d_lbp']
y_gas = res['sig_g_lbp']

## Check intensity fitting

In [ ]:
x = sim['r'] / au
obs = res['obs']

fig = plt.figure(constrained_layout=True, figsize=(12, 5))
gs = fig.add_gridspec(4, 4)
ax = fig.add_subplot(gs[:4, :2])
ax.set_xlabel(r'$r$ [au]')
ax.set_ylabel(r'$I_\nu$ [erg/(s cm$^2$ Hz sr)]')

for ilam in np.arange(len(lams)):
    
    # find the best match
    sampler = res['samplers'][ilam]
    discard = res['discards'][ilam]
    slice = sampler.lnprobability[:, discard:]
    idx = np.unravel_index(slice.argmax(), slice.shape)
    ln_best = slice[idx[0], idx[1]]
    p_best  = sampler.chain[:, discard:, :][idx[0], idx[1], :]
    _r_best = p_best[-1]
    print(f'r_best = {_r_best:.2g} au')
    
    txt = f'$\lambda = {lams[ilam] * 1e4:.0f}$ micron'
    
    # plot the model and determined dust line
    
    line, = ax.loglog(x, obs.I_nu[ilam], label=txt)
    ax.axvline(_r_best, c=line.get_color(), ls=':')
    
    # plot the logp evolution
    
    col = ilam//2
    row = ilam%2
    ax2 = fig.add_subplot(gs[2+row, 2+col])
    ax2.semilogy(-sampler.lnprobability.T, c='k', alpha=0.3);
    ax2.set_ylim(top=1e5)
    ax2.set_title(txt, {'color':line.get_color()})
    ax2.set_xlabel('iteration #')
    ax2.set_ylabel(r'$-\log P$')
    
    # overplot the fit
    
    if settings['fct_nr'] == 3:
        ym = dipsy.fortran.pwr2_logit(p_best, x)
    elif settings['fct_nr'] == 1:
        ym = dipsy.fortran.pwr1(p_best, x)
    ax.loglog(x, ym, c=line.get_color(), ls='--')
    

ax.legend()
ax.set_xlim(.05, 1e3)
ax.set_ylim(dipsy.fortran.crop, 1e2);

**Print the mass estimates**

In [ ]:
print('Gas:\n====')
print(f"trapz-based mass off by factor of {res['M_g_est'] / res['M_gas']:.2g}")
print(f"integral-based mass off by factor of {res['M_g_lbp'] / res['M_gas']:.2g}")
print(f"integral-masses avg off by factor of {res['M_g_med'] / res['M_gas']:.2g}")

print('\nDust:\n====')
print(f"trapz-based mass off by factor of {res['M_d_est'] / res['M_dust']:.2g}")
print(f"integral-based mass off by factor of {res['M_d_lbp'] / res['M_dust']:.2g}")
print(f"integral-masses avg off by factor of {res['M_d_med'] / res['M_dust']:.2g}")

**Check the particle size estimates**

In [ ]:
fig, ax = plt.subplots()

ax.loglog(sim['r'] / au, sim['a_dr'][it], '-', label='$a_\mathrm{drift}$')
ax.loglog(sim['r'] / au, sim['a_fr'][it], '-', label='$a_\mathrm{frag}$')

for _a, _r in zip(a, res['r_dust']):
    ax.axvline(_r, c='k', lw=0.5)
    ax.plot([_r], [_a], 'kx')
ax.legend()
ax.set_xlim(1e0, 1e3)
ax.set_ylim(1e-4, 1e0)
ax.set_xlabel(r'$r$ [au]')
ax.set_ylabel(r'$a$ [cm]');

## Check the LBP fit

In [ ]:
# plot the LBP sampler convergence

f, ax = plt.subplots(2, 2)
ax[0, 0].semilogy(-sampler_g.lnprobability.T);
ax[1, 0].semilogy(-sampler_d.lnprobability.T);

# plot the distribution of mass estimates

ax[0, 1].hist(np.log10(masses_g / res['M_gas']));
ax[1, 1].hist(np.log10(masses_d / res['M_dust']));

for _ax in ax[0]:
    _ax.set_title('gas')
for _ax in ax[1]:
    _ax.set_title('dust')
    
f.set_tight_layout(True)

In [ ]:
# calculate the dust and gas LBP fits with uncertainties

def wrapper(x, *args):
    return dipsy.fortran.lbp_profile(args, x)

y_d_min, y_d_max, y_d_array = estimator.get_sigma_area(sampler_d, wrapper, sim['r'], return_y=True)
y_g_min, y_g_max, y_g_array = estimator.get_sigma_area(sampler_g, wrapper, sim['r'], return_y=True)

# plot gas and dust surface densities

f, ax = plt.subplots()

ax.loglog(res['r_dust'], res['sig_g'], 'C0x', label='Powell, gas')
ax.loglog(res['r_dust'], res['sig_d'], 'C1x', label='drift estimate')

ax.loglog(sim['r'] / au, sim['sig_g'][it], 'C0-', label='simulation, gas')
ax.loglog(sim['r'] / au, sim['sig_d'][it], 'C1-', label='simulation, dust')

ax.loglog(sim['r'] / au, y_gas, 'C0--', label='LBP, gas')
ax.loglog(sim['r'] / au, y_dust, 'C1--', label='LBP, dust')

ax.fill_between(sim['r'] / au, y_g_min, y_g_max, fc='C0', alpha=0.2)
ax.fill_between(sim['r'] / au, y_d_min, y_d_max, fc='C1', alpha=0.2)

ax.legend()
ax.set_ylim(1e-5, 1e3)
ax.set_xlim(1e0, 5e2)
ax.set_xlabel(r'$r$ [au]')
ax.set_ylabel(r'$\Sigma$ [g/cm$^2$]')
fig.savefig('surface_densities.pdf', transparent=True)

## Check parameter guess

In [ ]:
obs = dipsy.get_observables(
    sim['r'],
    sim['sig_g'][it],
    sim['sig_d'][it],
    sim['a_max'][it],
    sim['T'][it],
    settings['opac'],
    lams,
)

In [ ]:
dipsy.fortran.crop=1e-10
ilam = -1

x = sim['r'] / au
y = obs.I_nu[ilam]
mask = x>1
x = x[mask]
y = y[mask]
p_guess, di = estimator.guess(x, y, 10, debug=True)

In [ ]:
f, ax = plt.subplots(2, 1, sharex=True, figsize=(8,8))

ax[0].loglog(x, y, 'k', lw=2)
for _p in p_guess:
    ax[0].loglog(x , dipsy.fortran.pwr2_logit(_p, x), 'g', alpha=0.5, lw=1)
    print('{:.2f}'.format(dipsy.fortran.lnp_pwr2_logit(_p, x, y)))

ax[1].semilogx(di['x'], di['exponent2'])

ax[1].axvline(di['r_dust'], c='r', lw=1, label=r'$r_\mathrm{dust}$')
ax[1].axvline(di['r_out'], c='k', lw=2, label=r'$r_\mathrm{out}$')
for _r in di['r_list']:
    ax[1].axvline(_r, c='k', lw=1, ls=':')
    
ax[1].legend()
ax[1].set_xlim(left=1);
ax[0].set_ylim(dipsy.fortran.crop, 1e4);